<a href="https://colab.research.google.com/github/minshyee/Section-4/blob/main/Get_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Get Data
## '방역패스', '백신패스' 에 대한 감성분석
### Naver News

In [1]:
!pip install Selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

import sys
sys.path.insert(0, '/usr/lib/chromium-browser/chromedriver')

     |████████████████████████████████| 958 kB 12.9 MB/s 
     |████████████████████████████████| 356 kB 73.0 MB/s 
     |████████████████████████████████| 138 kB 60.0 MB/s 
     |████████████████████████████████| 55 kB 5.2 MB/s 
     |████████████████████████████████| 3.6 MB 61.4 MB/s 
     |████████████████████████████████| 54 kB 4.1 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.8 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 http://archive.ubuntu.com/ubunt

In [3]:
from selenium import webdriver
chrome_options = webdriver.ChromeOptions() # 객체 가져오기

chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument("--single-process")
chrome_options.add_argument("--disable-dev-shm-usage")

# crawler_naver news

In [ ]:
# crawler_naver news
# Step 1. 네이버 > 검색어 입력 > 뉴스> 네이버뉴스 url 수집 > 저장 (naver_news_url.xlsx)
# Step 2. naver_news_url.xlsx 불러오기 > 제목, 날짜, 댓글, 좋아요 크롤링 > 저장 (naver_news_keyword.xlsx)

# pwd

### Step 0. 준비
import pandas as pd
import numpy as np

from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver import ActionChains as AC  # 웹 브라우저 동작
from tqdm import tqdm
from tqdm.notebook import tqdm
import re
from time import sleep
import time

### Step 1. 크롤링할 뉴스들 url 수집
# 검색어
keyword = "방역패스"
df_fin = pd.DataFrame()
# 크롬 웹브라우저 실행
driver = webdriver.Chrome('chromedriver', options=chrome_options)
for i in range(1,152,10):
  print(i)
  driver.get("https://search.naver.com/search.naver?where=news&sm=tab_jum&query={}&start={}".format(keyword,i))
  time.sleep(2)

# page1에서 네이버뉴스 url 수집하기
  things = driver.find_elements_by_link_text('네이버뉴스')  

  url_list = []
  for thing in things:
    url = thing.get_attribute('href')
    url_list.append(url)

  print(len(url_list))
  
# url_list

  # url_list 저장
  df = pd.DataFrame({"url":url_list})
  df_fin = pd.concat([df_fin,df])
# df.to_csv('naver_news_urls.xlsx')
df_fin



1


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:32: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead


26
11
29
21
21
31
16
41
14
51
19
61
12
71
25
81
13
91
17
101
31
111
22
121
16
131
14
141
13
151
23


,url
0,https://news.naver.com/main/read.naver?mode=LS...
1,https://news.naver.com/main/read.naver?mode=LS...
2,https://news.naver.com/main/read.naver?mode=LS...
3,https://news.naver.com/main/read.naver?mode=LS...
4,https://news.naver.com/main/read.naver?mode=LS...
...,...
18,https://news.naver.com/main/read.naver?mode=LS...
19,https://news.naver.com/main/read.naver?mode=LS...
20,https://news.naver.com/main/read.naver?mode=LS...
21,https://news.naver.com/main/read.naver?mode=LS...


In [ ]:
len(df_fin['url'].unique())

209

In [ ]:
df_fin = pd.DataFrame(df_fin['url'].drop_duplicates())

In [ ]:
df_fin.reset_index(drop=True,inplace=True)

In [ ]:
df_fin

,url
0,https://news.naver.com/main/read.naver?mode=LS...
1,https://news.naver.com/main/read.naver?mode=LS...
2,https://news.naver.com/main/read.naver?mode=LS...
3,https://news.naver.com/main/read.naver?mode=LS...
4,https://news.naver.com/main/read.naver?mode=LS...
...,...
204,https://news.naver.com/main/read.naver?mode=LS...
205,https://news.naver.com/main/read.naver?mode=LS...
206,https://news.naver.com/main/read.naver?mode=LS...
207,https://news.naver.com/main/read.naver?mode=LS...


In [ ]:

### Step 2. 전체 기사 for문으로 데이터 수집
# 저장해둔 url 불러오기
print(len(df_fin['url']))
# df['url']

number = len(df_fin['url'])
print(number)

dict = {}

# 페이지당 기사 수집
for i in tqdm(range(0, number)):   # len(df['url'])
    try:
        # 뉴스 크롬창 띄우기
        driver = webdriver.Chrome('chromedriver', options=chrome_options)
        driver.get(df_fin['url'][i])
        time.sleep(1)

        # 기사 데이터 수집
        title = driver.find_element_by_css_selector('.tts_head').text
        date = driver.find_element_by_css_selector('.t11').text
        up = driver.find_element_by_css_selector('.u_cnt._count').text
        like = driver.find_element_by_css_selector(".end_btn .u_likeit_list.good .u_likeit_list_count._count").text
        warm = driver.find_element_by_css_selector(".end_btn .u_likeit_list.warm .u_likeit_list_count._count").text
        sad = driver.find_element_by_css_selector(".end_btn .u_likeit_list.sad .u_likeit_list_count._count").text
        angry = driver.find_element_by_css_selector(".end_btn .u_likeit_list.angry .u_likeit_list_count._count").text
        want = driver.find_element_by_css_selector(".end_btn .u_likeit_list.want .u_likeit_list_count._count").text

        # 기사 댓글 갯수 
        review_count = int(driver.find_element_by_css_selector(".u_cbox_count").text.replace(',', ''))

        # 더보기 클릭 횟수
        moreview_num = review_count//20

        # 댓글 버튼 클릭
        driver.find_element_by_css_selector(".lo_txt").click()
        time.sleep(1)

        # 더보기 버튼 여러번 클릭하기
        k=0
        while k <= moreview_num:  # 더보기 횟수만큼 반복 
            try:
                driver.find_element_by_css_selector(".u_cbox_page_more").click()  # 더보기 버튼 클릭
                time.sleep(1)
                k = k+1
            except:        
                break   # 에러나면 클릭 반복문을 빠져나가라

        # review 수집하기
        review_list = []
        overlays1 = ".u_cbox_text_wrap"
        reviews = driver.find_elements_by_css_selector(overlays1)
        for review in tqdm(reviews):    
            review = review.text
            review_list.append(review)

        target_info = {}
        target_info['기사명'] = title
        target_info['날짜'] = date
        target_info['추천수'] = up
        target_info['좋아요'] = like
        target_info['훈훈해요'] = warm
        target_info['슬퍼요'] = sad
        target_info['화나요'] = angry
        target_info['후속기사 원해요'] = want
        target_info['댓글 수'] = len(review_list)
        target_info['댓글'] = review_list

        dict[i] = target_info
        
        print(title, '( 댓글 수:', len(review_list),')')

        driver.close()
        time.sleep(1)
        
    except:
        driver.close()
        continue

print(len(dict))
# dict

# 판다스화
import pandas as pd
result_df = pd.DataFrame.from_dict(dict, 'index')
result_df

# 엑셀 저장
# result_df.to_excel("naver_news_({}).xlsx".format(keyword))

209
209


  0%|          | 0/209 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:25: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:26: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
/usr/local/lib/python3.7/dis

  0%|          | 0/411 [00:00<?, ?it/s]

미접종 중증화율 3차접종보다 14배↑…"방역패스 효과 무시못해" ( 댓글 수: 411 )


...

美·유럽 방역패스 훑어보니…정부는 '도입' 국민은 '저항' 패턴 유사 ( 댓글 수: 122 )


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:25: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:26: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
/usr/local/lib/python3.7/dis

  0%|          | 0/86 [00:00<?, ?it/s]

윤석열 “마트 갈 자유도 제한…주먹구구식 방역패스 폐지해야” ( 댓글 수: 86 )
200


,기사명,날짜,추천수,좋아요,훈훈해요,슬퍼요,화나요,후속기사 원해요,댓글 수,댓글
0,"미접종 중증화율 3차접종보다 14배↑…""방역패스 효과 무시못해""",2022.01.11. 오후 12:17,"1,141",12,0,2,633,1,411,[https://news.naver.com/main/read.naver?mode=L...
1,"미접종 중증화율 3차접종보다 14배↑‥""방역패스 효과 무시못해""",2022.01.11. 오후 2:17,"1,441",60,1,0,395,3,194,[또 데이타 가공해서 지들 입맛에 유리하게 맞춘 데이타로 호도질.... Raw da...
2,"정부 ""방역패스 효과 무시못해…미접종자 중증화율 훨씬 높아""",2022.01.11. 오후 2:34,"2,010",1,0,2,344,0,119,"[법원 : ""접종률이 99%라면 코로나가 잡히나요?"" , 대께정부 : ""그건 아닙니..."
3,"윤석열, `방역 패스` 맹비난 ""비과학적이고 무리해""",2022.01.11. 오후 2:57,"1,740",465,0,0,55,2,166,"[무례하다 증말 ! 똥방역패스 시행해놓고 문통은 또 여행간다며?!, 맞는말 햇네 윤..."
4,"윤석열 ""방역패스 취지 이해 못할 바 아니지만 비과학적이고 무리한 측면 많아"" [뉴...",2022.01.11. 오후 4:07,336,36,0,0,12,0,12,[대안없이 지적만하기엔 대선이 너무 코앞 아닌가? 아직도 반사체에서 벗어나길 못하네...
...,...,...,...,...,...,...,...,...,...,...
204,"""과학적 사고 부족한 판사들"" 이재갑, '방역패스' 질의 법원 비판",2022.01.11. 오후 3:57,374,2,0,0,89,1,48,[이 미친인간 또 막말한다내가보기엔 판사가 의사인 너보다 더 과학적인거같은데... ...
205,“어딜 가든 다 불편하다”...방역패스 첫날 백화점·대형마트 입구부터 혼란,2022.01.10. 오후 2:43,48,34,2,7,"1,843",6,557,"[정권교체로 응징해야한다, 정부는 앞으로 들어올 백신재고가 걱정되서 그러시나요? 작..."
206,'혼밥' 허용한 나라 어딨나… 한국 방역패스는 '유연',2022.01.10. 오후 4:15,"1,753",18,1,0,459,2,172,"[아주 대단한 은혜를 베풀어 주는 것 처럼 이야기하네, 방역패스를 단속한단다. 단속..."
207,美·유럽 방역패스 훑어보니…정부는 '도입' 국민은 '저항' 패턴 유사,2022.01.10. 오후 4:40,125,6,0,2,489,2,122,[저중에 방역패스 우리만큼 심하게 하는 나라 없음 거짓선동기사 ㅋㅋㅋㅋㅋㅋㅋ95프로...


In [ ]:
result_df.reset_index(drop=True, inplace=True)
result_df

,기사명,날짜,추천수,좋아요,훈훈해요,슬퍼요,화나요,후속기사 원해요,댓글 수,댓글
0,"미접종 중증화율 3차접종보다 14배↑…""방역패스 효과 무시못해""",2022.01.11. 오후 12:17,"1,141",12,0,2,633,1,411,[https://news.naver.com/main/read.naver?mode=L...
1,"미접종 중증화율 3차접종보다 14배↑‥""방역패스 효과 무시못해""",2022.01.11. 오후 2:17,"1,441",60,1,0,395,3,194,[또 데이타 가공해서 지들 입맛에 유리하게 맞춘 데이타로 호도질.... Raw da...
2,"정부 ""방역패스 효과 무시못해…미접종자 중증화율 훨씬 높아""",2022.01.11. 오후 2:34,"2,010",1,0,2,344,0,119,"[법원 : ""접종률이 99%라면 코로나가 잡히나요?"" , 대께정부 : ""그건 아닙니..."
3,"윤석열, `방역 패스` 맹비난 ""비과학적이고 무리해""",2022.01.11. 오후 2:57,"1,740",465,0,0,55,2,166,"[무례하다 증말 ! 똥방역패스 시행해놓고 문통은 또 여행간다며?!, 맞는말 햇네 윤..."
4,"윤석열 ""방역패스 취지 이해 못할 바 아니지만 비과학적이고 무리한 측면 많아"" [뉴...",2022.01.11. 오후 4:07,336,36,0,0,12,0,12,[대안없이 지적만하기엔 대선이 너무 코앞 아닌가? 아직도 반사체에서 벗어나길 못하네...
...,...,...,...,...,...,...,...,...,...,...
195,"""과학적 사고 부족한 판사들"" 이재갑, '방역패스' 질의 법원 비판",2022.01.11. 오후 3:57,374,2,0,0,89,1,48,[이 미친인간 또 막말한다내가보기엔 판사가 의사인 너보다 더 과학적인거같은데... ...
196,“어딜 가든 다 불편하다”...방역패스 첫날 백화점·대형마트 입구부터 혼란,2022.01.10. 오후 2:43,48,34,2,7,"1,843",6,557,"[정권교체로 응징해야한다, 정부는 앞으로 들어올 백신재고가 걱정되서 그러시나요? 작..."
197,'혼밥' 허용한 나라 어딨나… 한국 방역패스는 '유연',2022.01.10. 오후 4:15,"1,753",18,1,0,459,2,172,"[아주 대단한 은혜를 베풀어 주는 것 처럼 이야기하네, 방역패스를 단속한단다. 단속..."
198,美·유럽 방역패스 훑어보니…정부는 '도입' 국민은 '저항' 패턴 유사,2022.01.10. 오후 4:40,125,6,0,2,489,2,122,[저중에 방역패스 우리만큼 심하게 하는 나라 없음 거짓선동기사 ㅋㅋㅋㅋㅋㅋㅋ95프로...


In [ ]:
cd /content/drive/MyDrive/AIBootCamp

/content/drive/MyDrive/AIBootCamp


In [ ]:
result_df.to_csv('news_vaccine_pass.csv', encoding="utf-8-sig")

In [ ]:
df = result_df.copy()

# 데이터 확인

In [ ]:
# 중복기사 삭제
df.drop_duplicates(['기사명'])

# 댓글 수 없는 기사 삭제
df[df['댓글 수'] != 0]

,기사명,날짜,추천수,좋아요,훈훈해요,슬퍼요,화나요,후속기사 원해요,댓글 수,댓글
0,"미접종 중증화율 3차접종보다 14배↑…""방역패스 효과 무시못해""",2022.01.11. 오후 12:17,"1,141",12,0,2,633,1,411,[https://news.naver.com/main/read.naver?mode=L...
1,"미접종 중증화율 3차접종보다 14배↑‥""방역패스 효과 무시못해""",2022.01.11. 오후 2:17,"1,441",60,1,0,395,3,194,[또 데이타 가공해서 지들 입맛에 유리하게 맞춘 데이타로 호도질.... Raw da...
2,"정부 ""방역패스 효과 무시못해…미접종자 중증화율 훨씬 높아""",2022.01.11. 오후 2:34,"2,010",1,0,2,344,0,119,"[법원 : ""접종률이 99%라면 코로나가 잡히나요?"" , 대께정부 : ""그건 아닙니..."
3,"윤석열, `방역 패스` 맹비난 ""비과학적이고 무리해""",2022.01.11. 오후 2:57,"1,740",465,0,0,55,2,166,"[무례하다 증말 ! 똥방역패스 시행해놓고 문통은 또 여행간다며?!, 맞는말 햇네 윤..."
4,"윤석열 ""방역패스 취지 이해 못할 바 아니지만 비과학적이고 무리한 측면 많아"" [뉴...",2022.01.11. 오후 4:07,336,36,0,0,12,0,12,[대안없이 지적만하기엔 대선이 너무 코앞 아닌가? 아직도 반사체에서 벗어나길 못하네...
...,...,...,...,...,...,...,...,...,...,...
195,"""과학적 사고 부족한 판사들"" 이재갑, '방역패스' 질의 법원 비판",2022.01.11. 오후 3:57,374,2,0,0,89,1,48,[이 미친인간 또 막말한다내가보기엔 판사가 의사인 너보다 더 과학적인거같은데... ...
196,“어딜 가든 다 불편하다”...방역패스 첫날 백화점·대형마트 입구부터 혼란,2022.01.10. 오후 2:43,48,34,2,7,"1,843",6,557,"[정권교체로 응징해야한다, 정부는 앞으로 들어올 백신재고가 걱정되서 그러시나요? 작..."
197,'혼밥' 허용한 나라 어딨나… 한국 방역패스는 '유연',2022.01.10. 오후 4:15,"1,753",18,1,0,459,2,172,"[아주 대단한 은혜를 베풀어 주는 것 처럼 이야기하네, 방역패스를 단속한단다. 단속..."
198,美·유럽 방역패스 훑어보니…정부는 '도입' 국민은 '저항' 패턴 유사,2022.01.10. 오후 4:40,125,6,0,2,489,2,122,[저중에 방역패스 우리만큼 심하게 하는 나라 없음 거짓선동기사 ㅋㅋㅋㅋㅋㅋㅋ95프로...


In [ ]:
df = df[df['댓글 수'] != 0]
df.reset_index(drop=True,inplace=True)
df

,기사명,날짜,추천수,좋아요,훈훈해요,슬퍼요,화나요,후속기사 원해요,댓글 수,댓글
0,"미접종 중증화율 3차접종보다 14배↑…""방역패스 효과 무시못해""",2022.01.11. 오후 12:17,"1,141",12,0,2,633,1,411,[https://news.naver.com/main/read.naver?mode=L...
1,"미접종 중증화율 3차접종보다 14배↑‥""방역패스 효과 무시못해""",2022.01.11. 오후 2:17,"1,441",60,1,0,395,3,194,[또 데이타 가공해서 지들 입맛에 유리하게 맞춘 데이타로 호도질.... Raw da...
2,"정부 ""방역패스 효과 무시못해…미접종자 중증화율 훨씬 높아""",2022.01.11. 오후 2:34,"2,010",1,0,2,344,0,119,"[법원 : ""접종률이 99%라면 코로나가 잡히나요?"" , 대께정부 : ""그건 아닙니..."
3,"윤석열, `방역 패스` 맹비난 ""비과학적이고 무리해""",2022.01.11. 오후 2:57,"1,740",465,0,0,55,2,166,"[무례하다 증말 ! 똥방역패스 시행해놓고 문통은 또 여행간다며?!, 맞는말 햇네 윤..."
4,"윤석열 ""방역패스 취지 이해 못할 바 아니지만 비과학적이고 무리한 측면 많아"" [뉴...",2022.01.11. 오후 4:07,336,36,0,0,12,0,12,[대안없이 지적만하기엔 대선이 너무 코앞 아닌가? 아직도 반사체에서 벗어나길 못하네...
...,...,...,...,...,...,...,...,...,...,...
163,"""과학적 사고 부족한 판사들"" 이재갑, '방역패스' 질의 법원 비판",2022.01.11. 오후 3:57,374,2,0,0,89,1,48,[이 미친인간 또 막말한다내가보기엔 판사가 의사인 너보다 더 과학적인거같은데... ...
164,“어딜 가든 다 불편하다”...방역패스 첫날 백화점·대형마트 입구부터 혼란,2022.01.10. 오후 2:43,48,34,2,7,"1,843",6,557,"[정권교체로 응징해야한다, 정부는 앞으로 들어올 백신재고가 걱정되서 그러시나요? 작..."
165,'혼밥' 허용한 나라 어딨나… 한국 방역패스는 '유연',2022.01.10. 오후 4:15,"1,753",18,1,0,459,2,172,"[아주 대단한 은혜를 베풀어 주는 것 처럼 이야기하네, 방역패스를 단속한단다. 단속..."
166,美·유럽 방역패스 훑어보니…정부는 '도입' 국민은 '저항' 패턴 유사,2022.01.10. 오후 4:40,125,6,0,2,489,2,122,[저중에 방역패스 우리만큼 심하게 하는 나라 없음 거짓선동기사 ㅋㅋㅋㅋㅋㅋㅋ95프로...


In [ ]:
# 총 사용 데이터
df['댓글 수'].sum()

55035

## 댓글 추출

In [ ]:
comments = []
for comment in df['댓글']:
  for c in comment:
    comments.append(c)

In [ ]:
len(comments)

55035

In [ ]:
raw_comments = pd.DataFrame(comments)
raw_comments

,0
0,https://news.naver.com/main/read.naver?mode=LS...
1,지금 거리두기 안하고 있냐? 타인 전파력은 접종자와 비접종자간 차이없다고 CDC도 ...
2,증증화율이 0.01%는 되나요? 14배면 0.14% 되겠네요? ㅋㅋㅋㅋ 증증화율 기...
3,부작용도 더 심하지 그건 왜 언급안하냐?
4,김서영기자 말장난으로 국민들 호도하지말고 백신 부작용 피해자가족을 위한 취재를 부탁한다.
...,...
55030,겁박하는 검사출신 답다. 지금 코로나형국에서 방역패스는 생명존중의 기본이다. 그런데...
55031,참 무식한 석열이
55032,"백신 맞아야 돌아다닌다. 니는어느나라 백성이냐, 뭐 니놈이 왕이다고? 미졌냐 니놈왕..."
55033,그니까 주먹구구식으로 반대만하지말고 과학적으로 체계적으로 이성적으로 대안을 제시하라...


# 삭제된 댓글 삭제

In [ ]:
raw_comments[raw_comments[0] == '작성자에 의해 삭제된 댓글입니다.']

,0
13,작성자에 의해 삭제된 댓글입니다.
56,작성자에 의해 삭제된 댓글입니다.
99,작성자에 의해 삭제된 댓글입니다.
104,작성자에 의해 삭제된 댓글입니다.
137,작성자에 의해 삭제된 댓글입니다.
...,...
55015,작성자에 의해 삭제된 댓글입니다.
55016,작성자에 의해 삭제된 댓글입니다.
55017,작성자에 의해 삭제된 댓글입니다.
55024,작성자에 의해 삭제된 댓글입니다.


In [ ]:
raw_comments[raw_comments[0] == '클린봇이 부적절한 표현을 감지한 댓글입니다.']

,0
159,클린봇이 부적절한 표현을 감지한 댓글입니다.
198,클린봇이 부적절한 표현을 감지한 댓글입니다.
210,클린봇이 부적절한 표현을 감지한 댓글입니다.
216,클린봇이 부적절한 표현을 감지한 댓글입니다.
221,클린봇이 부적절한 표현을 감지한 댓글입니다.
...,...
54847,클린봇이 부적절한 표현을 감지한 댓글입니다.
54876,클린봇이 부적절한 표현을 감지한 댓글입니다.
54883,클린봇이 부적절한 표현을 감지한 댓글입니다.
54901,클린봇이 부적절한 표현을 감지한 댓글입니다.


In [ ]:
raw_comments = raw_comments[raw_comments[0] != '작성자에 의해 삭제된 댓글입니다.']
raw_comments.reset_index(drop=True, inplace=True)

In [ ]:
raw_comments = raw_comments[raw_comments[0] != '클린봇이 부적절한 표현을 감지한 댓글입니다.']
raw_comments.reset_index(drop=True, inplace=True)

In [ ]:
raw_comments

,0
0,https://news.naver.com/main/read.naver?mode=LS...
1,지금 거리두기 안하고 있냐? 타인 전파력은 접종자와 비접종자간 차이없다고 CDC도 ...
2,증증화율이 0.01%는 되나요? 14배면 0.14% 되겠네요? ㅋㅋㅋㅋ 증증화율 기...
3,부작용도 더 심하지 그건 왜 언급안하냐?
4,김서영기자 말장난으로 국민들 호도하지말고 백신 부작용 피해자가족을 위한 취재를 부탁한다.
...,...
47084,겁박하는 검사출신 답다. 지금 코로나형국에서 방역패스는 생명존중의 기본이다. 그런데...
47085,참 무식한 석열이
47086,"백신 맞아야 돌아다닌다. 니는어느나라 백성이냐, 뭐 니놈이 왕이다고? 미졌냐 니놈왕..."
47087,그니까 주먹구구식으로 반대만하지말고 과학적으로 체계적으로 이성적으로 대안을 제시하라...


In [ ]:
raw_comments.to_csv('raw_comments_2.csv' , encoding="utf-8-sig")

In [78]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(raw_comments, test_size=0.7)

In [79]:
train.shape

(14126, 1)

In [81]:
train.to_csv('comment_train.csv', encoding="utf-8-sig")
test.to_csv('comment_test.csv', encoding="utf-8-sig")